In [1]:
import math
from scipy import stats as st

cdfInterval = 0.99

def G2StatisticsIndepTest(indpTuple):
    
    xCombCnt = indpTuple.xVarInfo.noOfVals
    yCombCnt = indpTuple.yVarInfo.noOfVals
    
    totalVal = 0.0
    for zVal, xyValMap in indpTuple.indepXyStat.iteritems():
        for xyValKey, cntVal in xyValMap.iteritems():
            xValKey, yValKey = xyValKey.split("##")
            
            Mx = float(indpTuple.indepXStat[zVal][xValKey])
            My = float(indpTuple.indepYStat[zVal][yValKey])
            Mxy = float(cntVal)
            M = float(indpTuple.indepZStat[zVal])
        
            
            #print Mx, My, Mxy
        
            if (Mx>0.0 and My>0.0 and Mxy>0.0):
                totalVal = totalVal + (Mxy * math.log(Mxy * M/ (Mx * My)))
    
    totalVal = 2 * totalVal
    print "Total Val is ", totalVal
    threshold = st.chi2.ppf(cdfInterval, df= (xCombCnt-1) * (yCombCnt-1) )
    print "Threshold is ", threshold
    if totalVal < threshold:
        return True
    return False    

In [2]:
import math

cdfInterval = 0.99

def G2StatisticsZIndepTest(indpTuple):
    
    xCombCnt = indpTuple.xVarInfo.noOfVals
    yCombCnt = indpTuple.yVarInfo.noOfVals
    
    zCombCnt = 1.0
    for zVal in indpTuple.zListVarInfo:
        zComCnt = zCombCnt * zVal.noOfVals
    
    totalVal = 0.0
    
    for zVal, xyValMap in indpTuple.indepXyStat.iteritems():
        for xyValKey, cntVal in xyValMap.iteritems():
            xValKey, yValKey = xyValKey.split("##")
            
            
            Mxz = float(indpTuple.indepXStat[zVal][xValKey])
            Myz = float(indpTuple.indepYStat[zVal][yValKey])
            Mxyz = float(cntVal)
            Mz = float(indpTuple.indepZStat[zVal])
            

            if (Mxz>0.0 and Myz>0.0 and Mxyz>0.0):
                totalVal = totalVal + (Mxyz * math.log((Mxyz * Mz) / (Mxz * Myz)))

    totalVal =  2 * totalVal
    print "Total Val is ", totalVal
    threshold = st.chi2.ppf(cdfInterval, df=  zCombCnt * (xCombCnt-1) * (yCombCnt-1) )
    print "Threshold is ", threshold
    if totalVal < threshold:
        return True
    return False    

In [3]:
class lexCombIdx(object):
    def __init__(self, countOfBatchSize,lSize):    
        self.batchSize = countOfBatchSize
        self.listSize = lSize
        self.idxLimit = lSize-1
        self.idxList = []
        self.isEmpty = False
        for idx in range(0,self.batchSize):
            self.idxList.append(idx)
        
        
    def getNextCombIndex(self):
        retVal = self.idxList
        self.idxList = []
        
        for lVal in retVal:
            self.idxList.append(lVal)
        
        while len(self.idxList) > 0:
            lastVal = self.idxList.pop()
            lastVal= lastVal+1
            currSize = len(self.idxList)
            finalIdx = lastVal + self.batchSize - currSize -1
            if(lastVal<self.listSize and finalIdx < self.listSize):
                for idx in range(lastVal, lastVal + self.batchSize - currSize):
                    self.idxList.append(idx)
                
                break
        
        
        return retVal

In [4]:
class varInfo(object):
    def __init__(self, vName, idInData, nVals, vList):
        self.name = vName
        self.idxInData = idInData
        self.noOfVals = nVals
        self.valList = vList
        
    

In [5]:
class indepTuple(object):
    def __init__(self, xVInfo, yVInfo, nbrList,zListVInfo):
        self.xVarInfo = xVInfo
        self.yVarInfo = yVInfo
        self.nbrs = nbrList
        self.zListVarInfo = zListVInfo
        self.indepStat = {}
        self.indepXyStat = {}
        self.indepXStat = {}
        self.indepYStat = {}
        self.indepZStat = {}
        self.isIndep = False
        

In [6]:
import pandas as pd
class inputDataObj(object):
    def __init__(self, fName, headerFileName):
        self.fileName = fName
        self.headerInfoFileName = headerFileName
        self.headerNames = ""
        self.varInfoList = []
        self.noOfvars = 0
        self.idxToVar = {}
        self.fileDf = None
        
        df = pd.read_csv(self.headerInfoFileName, sep="#")
        print "New Test"
        for idx,row in df.iterrows(): 
            valList = list(row['values'].split(","))
            colName = row['attribute']
            varVal = varInfo(colName, idx, len(valList), valList)
            self.varInfoList.append(varVal)
            self.headerNames = self.headerNames + colName + ","
            self.idxToVar[idx] = varVal
        
        
        self.headerNames = self.headerNames[0 : (len(self.headerNames)-1)]
        self.noOfvars = len(self.varInfoList)
        
    #def getCountStatForASingleCombo(self, xVarInfo, yVarInfo, zListVarInfo):
    #    pass
    
    def getDataObj(self):
        pass
    

In [29]:
class singleSystemFileObjPC(inputDataObj):
    def getDataObj(self):
        if self.fileDf is None:
            df = pd.read_csv(self.fileName, header=0, names = self.headerNames.split(","))
            self.fileDf = df
        else:
            df = self.fileDf
            
        return df

In [15]:
class distSystemFileObjPC(inputDataObj):
    def __init__(self, fName, headerFileName, hdfsIp):
        inputDataObj.__init__(fName, headerFileName)
        conf = SparkConf().setAppName("DistPcStableAlgo")
        sc = SparkContext(conf=conf)
        self.hdfs_ip = hdfsIp #'hdfs://10.88.6.167:8020'
        self.modelDataPath = hdfs_ip + '/' + self.fileName
        self.rdd = sc.textFile(self.modelDataPath).map(lambda line: line.split(","))
        self.listOfIndepTuple = None
        
    def getDataObj(self):
        return self.rdd

In [9]:
class statGather():
    def getCountStatForBatch(self, dataObj,listOfIndepTuple):
        pass

In [25]:
import pandas as pd

class singleSystemStatGathering(statGather):
    
    def getCountStatForBatch(self, dataObj,listOfIndepTuple):
        
        
        df=dataObj.getDataObj()
        
        
        for idx,row in df.iterrows():
            if idx%1000 == 0:
                print "Idx Value is " + str(idx)
            for indpTup in listOfIndepTuple:
                zVal = ""
                for zVarInfo in indpTup.zListVarInfo:
                    zVal = zVal + str(row[zVarInfo.idxInData]) + ("#")
                
                #indpTup.indepXyStat = {}
                #indpTup.indepXStat = {}
                #indpTup.indepYStat = {}
                #indpTup.indepZStat = {}
                
                if zVal not in indpTup.indepXyStat:
                    indpTup.indepXyStat[zVal] = {}
                    
                if zVal not in indpTup.indepXStat:
                    indpTup.indepXStat[zVal] = {}
                    
                if zVal not in indpTup.indepYStat:
                    indpTup.indepYStat[zVal] = {}
                    
                if zVal not in indpTup.indepZStat:
                    indpTup.indepZStat[zVal] = 0.0
                
                xVal = str(row[indpTup.xVarInfo.idxInData])
                yVal = str(row[indpTup.yVarInfo.idxInData])
                xyVal = str(row[indpTup.xVarInfo.idxInData]) + "##" + str(row[indpTup.yVarInfo.idxInData]) 
                
                if xyVal not in indpTup.indepXyStat[zVal]:
                    indpTup.indepXyStat[zVal][xyVal] = 0.0
                
                if xVal not in indpTup.indepXStat[zVal]:
                    indpTup.indepXStat[zVal][xVal] = 0.0
                    
                if yVal not in indpTup.indepYStat[zVal]:
                    indpTup.indepYStat[zVal][yVal] = 0.0
                
                cnt = indpTup.indepXyStat[zVal][xyVal]
                cnt = cnt + 1.0
                indpTup.indepXyStat[zVal][xyVal] = cnt
                
                cnt = indpTup.indepXStat[zVal][xVal]
                cnt = cnt + 1.0
                indpTup.indepXStat[zVal][xVal] = cnt
                
                cnt = indpTup.indepYStat[zVal][yVal]
                cnt = cnt + 1.0
                indpTup.indepYStat[zVal][yVal] = cnt
                
                cnt = indpTup.indepZStat[zVal]
                cnt = cnt + 1.0
                indpTup.indepZStat[zVal] = cnt

In [26]:
class distSystemStatGth(statGather):
    
    def __init__(self):
        self.listOfIndepTuple = None
        
    def getCountStatForBatch(self, dataObj, listOfIndepTuple):
        self.listOfIndepTuple = listOfIndepTuple
        
        resultMap = dataObj.getDataObj().mapPartitionsWithIndex(self.FuncComputeStats)
        
        finalResult = resultMap.reduceByKey(lambda accum, n: accum + n) 
        
        mapIndepTuple = {}
        
        for indpTup in listOfIndepTuple:
            zVal = ""
            for zVarInfo in indpTup.zListVarInfo:
                zVal = zVal + str(zVarInfo.idxInData) + "#"

            finalKeyVal =  str(indpTup.xVarInfo.idxInData) + "_:_" + str(indpTup.yVarInfo.idxInData) + "_:_" + zVal
            
            mapIndepTuple[finalKeyVal] = indpTup
        
        
        
        for key, val in finalResult.collect():
            splitVal = key.split(":::")
            indpKey = splitVal[0]
            indpTup = mapIndepTuple[indpKey]
            
            zValIndpStatKey = splitVal[1].split("--")[0]
            valIndpStatKey = splitVal[1].split("--")[1]
            
            if zValIndpStatKey not in indpTup.indepZStat:
                indpTup.indepZStat[zValIndpStatKey] =0.0
                
            
            
            if zValIndpStatKey not in indpTup.indepXStat:
                indpTup.indepXStat[zValIndpStatKey]={}
                
            if zValIndpStatKey not in indpTup.indepYStat:
                indpTup.indepYStat[zValIndpStatKey] = {}

            if zValIndpStatKey not in indpTup.indepXyStat:
                indpTup.indepXyStat[zValIndpStatKey]={}
                
            if valIndpStatKey.startswith("x#"):
                valIndpStatKey=valIndpStatKey.replace("x#","")
                indpTup.indepXStat[zValIndpStatKey][valIndpStatKey] = val
            elif valIndpStatKey.startswith("y#"):
                valIndpStatKey=valIndpStatKey.replace("y#", "")
                indpTup.indepYStat[zValIndpStatKey][valIndpStatKey] = val
            elif valIndpStatKey.startswith("xy#"):
                valIndpStatKey=valIndpStatKey.replace("xy#","")
                indpTup.indepXyStat[zValIndpStatKey][valIndpStatKey] = val
                indpTup.indepZStat[zValIndpStatKey] = indpTup.indepZStat[zValIndpStatKey] + val
            else:
                print("Errorrrrrrrr Major Errorrrrrr")
                
            
                
        self.listOfIndepTuple = None
        
    def FuncComputeStats(self, splitIndex, iterator):
        for row in iterator:
            for indpTup in self.listOfIndepTuple:
                zVal = ""
                for zVarInfo in indpTup.zListVarInfo:
                    zVal = zVal + str(row[zVarInfo.idxInData]) + "#"
                
                
                if zVal not in indpTup.indepStat:
                    indpTup.indepStat[zVal] = {}
                    
                xVal = "x#" + str(row[indpTup.xVarInfo.idxInData])
                yVal = "y#" + str(row[indpTup.yVarInfo.idxInData]) 
                xyVal = "xy#" + str(row[indpTup.xVarInfo.idxInData]) + "##" + str(row[indpTup.yVarInfo.idxInData]) 
                
                
                
                if xyVal not in indpTup.indepStat[zVal]:
                    indpTup.indepStat[zVal][xyVal] = 0.0
                
                cnt = indpTup.indepStat[zVal][xyVal]
                cnt = cnt + 1.0
                indpTup.indepStat[zVal][xyVal] = cnt
                
                
                if xVal not in indpTup.indepStat[zVal]:
                    indpTup.indepStat[zVal][xVal] = 0.0
                
                cnt = indpTup.indepStat[zVal][xVal]
                cnt = cnt + 1.0
                indpTup.indepStat[zVal][xVal] = cnt
                
                if yVal not in indpTup.indepStat[zVal]:
                    indpTup.indepStat[zVal][yVal] = 0.0
                
                cnt = indpTup.indepStat[zVal][yVal]
                cnt = cnt + 1.0
                indpTup.indepStat[zVal][yVal] = cnt
        
        
        listVals = []
        
        for indpTup in self.listOfIndepTuple:
            
            zVal = ""
            for zVarInfo in indpTup.zListVarInfo:
                zVal = zVal + str(zVarInfo.idxInData) + "#"

            finalKeyVal =  str(indpTup.xVarInfo.idxInData) + "_:_" + str(indpTup.yVarInfo.idxInData) + "_:_" + zVal
            
            for key1, val1 in indpTup.indepStat.iteritems():
                
                for key2, val2 in val1.iteritems():
                    outKey =  finalKeyVal + ":::" + str(key1) + "--" + str(key2)   
                    listVals.append((outKey, val2))
        
        return listVals
        

In [19]:
import numpy as np


class PcAlgo(object):
    def __init__(self, inData, statGthObj):
        self.inDataObj = inData
        self.statGatherObj = statGthObj
        self.feaSizeList = inData.noOfvars
        self.adjMat = np.ones((self.feaSizeList, self.feaSizeList), dtype=np.int8)
        self.batchSize = 500
    
    def algo(self):
        #obsCntList = range(0, self.feaSizeList-1)
        obsCntList= [0,1,2,3,4] # TO be removed before final run
        varIdList = range(0, self.feaSizeList)
        
        
        for obsCnt in obsCntList:
            print "ObservedCount Size is " + str(obsCnt)
            varNbrMap = {}
            nbrPairList = []
            lexicoGraphicalNextMap = {}
            for varId in varIdList:
                varNbrs = []
                varNbrMap[varId] = varNbrs
                for nbrId in varIdList:
                    if varId != nbrId:
                        if self.adjMat[varId][nbrId] == 1:
                            varNbrs.append(nbrId)
                    

            
            
            for varId, nbrS in varNbrMap.iteritems():
                if len(nbrS) >= obsCnt:
                    
                    for nbrId in nbrS :
                        if nbrId > varId:
                            lexKey = str(varId) + "_" + str(nbrId)
                            lexicoGraphicalNextMap[lexKey] = lexCombIdx(obsCnt, len(nbrS)-1)
             
            while(True):

                listOfIndepTuple = []
                while(len(lexicoGraphicalNextMap) > 0 and len(listOfIndepTuple) < self.batchSize):
                    rmvKeyIndepPair = []

                    for keyIndepPair, valLexObj in lexicoGraphicalNextMap.iteritems():
                        varsIdx = keyIndepPair.split("_")
                    
                        if self.adjMat[int(varsIdx[0])][int(varsIdx[1])] == 0:
                            rmvKeyIndepPair.append(keyIndepPair)
                            continue
        
                        xVarInfo = self.inDataObj.idxToVar[int(varsIdx[0])]
                        yVarInfo = self.inDataObj.idxToVar[int(varsIdx[1])]
                        
                        if(obsCnt==0):
                            zVarInfoList = []
                            
                            indpTp = indepTuple(xVarInfo, yVarInfo, None, zVarInfoList)
                            
                            listOfIndepTuple.append(indpTp)
                            
                            rmvKeyIndepPair.append(keyIndepPair)
                            
                            if len(listOfIndepTuple) >= self.batchSize:
                                break
                        else:
                            nbrSForIndep =[]
                            for val in varNbrMap[xVarInfo.idxInData]:
                                if val != yVarInfo.idxInData:
                                    nbrSForIndep.append(val)

                            if ((nbrSForIndep is None) or (len(nbrSForIndep) < obsCnt)):
                                rmvKeyIndepPair.append(keyIndepPair)
                                continue

                            listOfNbrIdx = valLexObj.getNextCombIndex()
                            if(len(listOfNbrIdx)==0):
                                rmvKeyIndepPair.append(keyIndepPair)
                                print "Indep Pairs Exhausted"
                                continue

                            zVarInfoList = []


                            for idx in listOfNbrIdx:
                                zVarInfoList.append(self.inDataObj.idxToVar[nbrSForIndep[idx]])

                            indpTp = indepTuple(xVarInfo, yVarInfo, nbrSForIndep, zVarInfoList)

                            #Print statements to be removed
                            #print "Xvar " + indpTp.xVarInfo.name
                            #print "YVar " + indpTp.yVarInfo.name 

                            listOfIndepTuple.append(indpTp)

                            if len(listOfIndepTuple) >= self.batchSize:
                                break

                    for rmvKey in rmvKeyIndepPair:
                        lexicoGraphicalNextMap.pop(rmvKey, None)

                        
                        
                print "Size of ListIndepTuple " + str(len(listOfIndepTuple))
                self.statGatherObj.getCountStatForBatch(self.inDataObj, listOfIndepTuple)

                
                for indpTuple in listOfIndepTuple:
                    indepResult = False

                    if obsCnt == 0:
                        indepResult = G2StatisticsIndepTest(indpTuple)

                        #Print statements to be removed
                        print "Xvar " + indpTuple.xVarInfo.name
                        print "YVar " + indpTuple.yVarInfo.name

                        print "IndepResult " + str(indepResult)

                    else :
                        indepResult = G2StatisticsZIndepTest(indpTuple)
                        print "Xvar " + indpTuple.xVarInfo.name
                        print "YVar " + indpTuple.yVarInfo.name
                        print "ZList" + str(indpTuple.zListVarInfo)

                        print "IndepResult " + str(indepResult)

                    indpTuple.isIndep = indepResult
                    if indepResult == True:
                        self.adjMat[indpTuple.xVarInfo.idxInData][indpTuple.yVarInfo.idxInData] = 0
                        self.adjMat[indpTuple.yVarInfo.idxInData][indpTuple.xVarInfo.idxInData] = 0

                listOfIndepTuple=[]
                
                if len(lexicoGraphicalNextMap) == 0:
                    break

            

In [30]:
fileDataObj = singleSystemFileObjPC("D:\\Study\\DataScience\\FlightDelay\\ORDDiscrete2008DebugData.csv", "D:\\Study\\DataScience\\FlightDelay\\headerInfo.txt")
statGthObj = singleSystemStatGathering()
pcInst = PcAlgo(fileDataObj, statGthObj)

New Test


In [32]:
#pcInst.algo()
print 'Hi'

Hi


In [ ]:
fileDataObj = distSystemFileObjPC("/Development/FD/ORDDiscrete2008DebugData.csv", "/ftp1/test/FD/headerInfo.txt", "hdfs://10.184.8.22:8020")
statGthObj = distSystemStatGth()
pcInst = PcAlgo(fileDataObj, statGthObj)
pcInst.algo()

In [45]:
listVal = [1,2,3,4,5]
print listVal.remove(2)


None


In [5]:
import pandas as pd

df = pd.read_csv("D:\\Study\\DataScience\\FlightDelay\\ORDDiscrete2008.csv", header=None)

idxToBeRemoved = []

idxToBeRemoved.extend(range(1000, 11000))
idxToBeRemoved.extend(range(12000, 22000))
idxToBeRemoved.extend(range(23000, 33000))
idxToBeRemoved.extend(range(34000, 44000))
idxToBeRemoved.extend(range(45000, 55000))
idxToBeRemoved.extend(range(56000, 66000))
idxToBeRemoved.extend(range(67000, 77000))
idxToBeRemoved.extend(range(78000, 88000))
idxToBeRemoved.extend(range(89000, 99000))
idxToBeRemoved.extend(range(101000, 111000))
idxToBeRemoved.extend(range(112000, 122000))
idxToBeRemoved.extend(range(123000, 133000))
idxToBeRemoved.extend(range(134000, 144000))
idxToBeRemoved.extend(range(145000, 155000))
idxToBeRemoved.extend(range(156000, 166000))
idxToBeRemoved.extend(range(167000, 177000))
idxToBeRemoved.extend(range(178000, 188000))
idxToBeRemoved.extend(range(189000, 199000))
idxToBeRemoved.extend(range(201000, 211000))
idxToBeRemoved.extend(range(212000, 222000))
idxToBeRemoved.extend(range(223000, 233000))
idxToBeRemoved.extend(range(234000, 244000))
idxToBeRemoved.extend(range(245000, 255000))
idxToBeRemoved.extend(range(256000, 266000))
idxToBeRemoved.extend(range(267000, 277000))
idxToBeRemoved.extend(range(278000, 288000))

df1 = df.drop(idxToBeRemoved, axis=0)

df1.to_csv("D:\\Study\\DataScience\\FlightDelay\\ORDDiscrete2008DebugData.csv", index=False)

In [120]:
varIdList = range(0, pcInst.feaSizeList)
varNbrMap={}
for varId in varIdList:
    varNbrs = []
    varNbrMap[varId] = varNbrs
    for nbrId in varIdList:
        if varId != nbrId:
            if pcInst.adjMat[varId][nbrId] == 1:
                varNbrs.append(nbrId)

print varNbrMap        

{0: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 1: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 2: [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3: [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 4: [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 5: [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 6: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 7: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 8: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 9: [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 10: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 11: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20], 12: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20], 13: [0, 1, 

In [33]:


df = pd.read_csv("D:\\Study\\DataScience\\FlightDelay\\headerInfo.txt", sep="#")

print df.columns

for idx, row in df.iterrows():
    print list(row['values'].split(","))

Index([u'attribute', u'values'], dtype='object')
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
['0', '1', '2', '3']
['0', '1', '2', '3', '4']
['0', '1']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
['0', '1', '2', '3']
['0', '1', '2', '3']
['0', '1', '2', '3']
['0', '1', '2']
['0', '1']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
['0', '1', '2', '3', '4']
['0', '1', '2', '3']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']
['0', '1']
['0', '1', '2', '3']
['0', '1', '2',